# Text Classification Using Transformer Networks (BERT)

Some initialization:

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

In [1]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

In [2]:
def read_dota_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["text","target"]]

In [3]:
# labels = open('classes.txt').read().splitlines()
labels = ["0","1"]
data = read_dota_data('tagged-data.csv')
print(labels)
data = data.rename(columns={"target": "label"})
data['label'] = data['label'].replace(2,1)
data

['0', '1']


,text,label
0,COMMEND ME TY,0
1,sorry nex,0
2,what is the best soup?,0
3,man that silence on axe,0
4,not coming into play,0
...,...,...
3262,"wt?f?asfU JGOFIDLK,YH",1
3263,you must really suck,1
3264,YOU HAVE IDIOT PLAYER,1
3265,SUPER IDIOT,1


In [4]:
from sklearn.model_selection import train_test_split

train_df, eval_and_test_df = train_test_split(data, train_size=0.8, random_state=seed)
eval_df, test_df = train_test_split(eval_and_test_df, train_size=0.5, random_state=seed)
train_df.reset_index(inplace=True, drop=True)
eval_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'train rows: {len(train_df.index):,}')
print(f'eval rows: {len(eval_df.index):,}')
print(f'test rows: {len(test_df.index):,}')

train rows: 2,613
eval rows: 327
test rows: 327


In [5]:
eval_df

,text,label
0,youtube.com/mirocj01 -----> Divine crusher,0
1,u done nothing,0
2,end please,0
3,WHO WILL SUCK DICK AGTER,1
4,sry only noobs here,1
...,...,...
322,you are not pro btw,0
323,just report them,0
324,He asked for a pause to recconect,0
325,and suck my dick,1


In [6]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df)
ds['validation'] = Dataset.from_pandas(eval_df)
ds['test'] = Dataset.from_pandas(test_df)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2613
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 327
    })
})

Tokenize the texts:

In [7]:
from transformers import AutoTokenizer

transformer_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name,use_fast=True)

In [8]:
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/2613 [00:00<?, ? examples/s]

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 1108, 4106, 2286, 24438, 1186, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
1,1,"[101, 17480, 1801, 1518, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
2,1,"[101, 1143, 1324, 117, 1714, 11348, 1111, 2243...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,0,"[101, 124, 1110, 1268, 1397, 1106, 125, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
4,0,"[101, 1192, 1132, 1226, 1104, 1115, 1679, 1358...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
2608,0,"[101, 3869, 136, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2609,1,"[101, 1228, 28015, 1116, 10696, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
2610,0,"[101, 1195, 1136, 1138, 126, 6485, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
2611,0,"[101, 1198, 2999, 1342, 1155, 1444, 3858, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"


Create the transformer model:

In [9]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [10]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    num_labels=len(labels),
    # num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config).to(device)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create the trainer object and train:

In [11]:
from transformers import TrainingArguments

num_epochs = 10
batch_size = 24
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    weight_decay=weight_decay,
    fp16=True
)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

/tmp/ipykernel_5652/485909320.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.172659,0.948012
2,No log,0.227145,0.923547
3,No log,0.460075,0.920489
4,No log,0.356314,0.932722
5,0.131400,0.386347,0.944954
6,0.131400,0.374154,0.944954
7,0.131400,0.431227,0.941896
8,0.131400,0.456357,0.941896
9,0.131400,0.466664,0.941896
10,0.006800,0.471219,0.941896


TrainOutput(global_step=1090, training_loss=0.06350526098811299, metrics={'train_runtime': 133.7753, 'train_samples_per_second': 195.328, 'train_steps_per_second': 8.148, 'total_flos': 261021069691740.0, 'train_loss': 0.06350526098811299, 'epoch': 10.0})

Evaluate on the test partition:

In [15]:
test_ds = ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['text'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds.to_pandas()

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,1,"[101, 6100, 2592, 1142, 1266, 26144, 3676, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 7642, 5773, 102]","[0, 0, 0, 0]","[1, 1, 1, 1]"
2,0,"[101, 1710, 136, 136, 136, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
3,0,"[101, 189, 1548, 18257, 2956, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
4,1,"[101, 1534, 1183, 2087, 1665, 21569, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
322,0,"[101, 1231, 1643, 171, 1324, 1126, 23609, 8484...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
323,1,"[101, 3919, 14084, 4894, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
324,0,"[101, 126, 1904, 3932, 1297, 13894, 5815, 1162...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
325,0,"[101, 1119, 1116, 1435, 1158, 1171, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"


In [16]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[-5.2617188e+00,  6.3085938e+00],
       [ 5.3945312e+00, -5.5351562e+00],
       [ 5.6953125e+00, -6.3125000e+00],
       [ 5.0820312e+00, -5.6445312e+00],
       [ 4.5781250e+00, -4.8320312e+00],
       [ 5.5468750e+00, -6.2265625e+00],
       [-4.3281250e+00,  4.1875000e+00],
       [-5.2343750e+00,  6.2539062e+00],
       [ 4.7265625e+00, -5.7031250e+00],
       [ 5.2539062e+00, -5.9062500e+00],
       [-4.6562500e+00,  5.1914062e+00],
       [ 5.1835938e+00, -6.1015625e+00],
       [-5.1210938e+00,  6.3046875e+00],
       [ 5.3515625e+00, -5.9648438e+00],
       [-4.9843750e+00,  6.0820312e+00],
       [ 5.4101562e+00, -6.2929688e+00],
       [-5.1406250e+00,  6.2031250e+00],
       [-5.2460938e+00,  6.2031250e+00],
       [ 5.4687500e+00, -6.3632812e+00],
       [-1.9716797e+00,  1.4345703e+00],
       [-3.0878906e+00,  2.7695312e+00],
       [ 5.5742188e+00, -5.9882812e+00],
       [-4.5546875e+00,  4.7500000e+00],
       [-4.6679688e+00,  5.1

In [18]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic","toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.96      0.96      0.96       201
       toxic       0.93      0.93      0.93       126

    accuracy                           0.94       327
   macro avg       0.94      0.94      0.94       327
weighted avg       0.94      0.94      0.94       327



In [93]:
# Sample DataFrame (make sure your real DataFrame has the correct structure)
cols = ["text", "label"]
data = [["soup soup soup soup ",0],
        ["I hope you outlive your children",1],
        ["I hope you hurt your children",1],
        ["I hope you kill your children",1],
        ["I hope you beat your children",1],
        ["I hope you eat your children",1],
        ["kids soup",0],
        ["soup kids",0],
        ["children stew ",0],
        ["I hope you die",1],
        ["I hope your kids die",1],
        ["I hope your kids water", 0],
        ["I hope your kids soup", 0],
        ["I hope your kids win",0],
        ["I hope your kids lose", 1],
        ["I hope your kids",0],
        ["Can you lose already?",0],
        ["die udyr", 1],
        ["die kids", 1],
        ["I hope your lose",0],
        ["I hope your succeed",1],
        ["kisll him KILL him",1],
        ["NUKE HIS ASS",0]
       ]
test = pd.DataFrame(data, columns = cols)

# Convert the DataFrame to a Dataset
test_dataset = Dataset.from_pandas(test)
#test_dataset.reset_index(inplace=True, drop=True)

dota_ds=DatasetDict()
dota_ds['test'] = Dataset.from_pandas(test)
#dota_ds.reset_index(inplace=True, drop=True)


# Now, map the tokenization function to the dataset
# Since your key for the dataset is 'test', make sure you access 'test' from dota_ds
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    # remove_columns=['message'],  # Removing 'message' column after tokenization
)

# Convert to pandas DataFrame (for inspection)
test_ds_dota_df = test_ds_dota.to_pandas()

# Output the tokenized dataset
# print(test_ds_dota_df)


# Make predictions
output_dota = trainer.predict(test_ds_dota)

# Print or inspect the output
# print(output_dota)

output_dota.predictions

chats = test_dataset["text"]
labels = np.argmax(output_dota.predictions, axis=-1)
pd.DataFrame(labels, chats)

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

,0
soup soup soup soup,0
I hope you outlive your children,0
I hope you hurt your children,0
I hope you kill your children,1
I hope you beat your children,1
I hope you eat your children,1
kids soup,1
soup kids,1
children stew,0
I hope you die,0
